In [1]:
from nn_packages import *
from io_functions import *
import numpy as np
import root_numpy as rnp
import os
import sys
import re
import glob
import h5py
import numpy as np
#import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Activation,Input, Dense, Dropout, merge
from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN not available)
Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


/usr/local/lib/python2.7/dist-packages/root_numpy/__init__.py:35: RuntimeWarning: numpy 1.11.1 is currently installed but you installed root_numpy against numpy 1.11.0. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


Welcome to ROOTaaS 6.06/04


In [2]:
def loadmodel(name, weights = False):
    json_file = open('%s.json'%name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    #load weights into new model
    if weights==True:
        model.load_weights('%s.h5'%name)
    #print (model.summary())
    print("Loaded model from disk")
    return model

def savemodel(model,name="neural network"):

    model_name = name
    model.summary()
    #model.save_weights('%s.h5'%model_name, overwrite=True)
    model_json = model.to_json()
    with open("%s.json"%model_name, "w") as json_file:
        json_file.write(model_json)
        
def savelosses(hist, name="neural network"):    
    loss = np.array(hist.history['loss'])
    valoss = np.array(hist.history['val_loss'])
    f = h5py.File("%s_losses.h5"%name,"w")
    f.create_dataset('loss',data=loss)
    f.create_dataset('val_loss',data=valoss)
    f.close()

SyntaxError: non-keyword arg after keyword arg (<ipython-input-5-36b416ed84eb>, line 1)

In [ ]:
input1 = Input(shape = (1, 20, 20, 25))
model1 = Convolution3D(3, 4, 4, 5, input_shape = (1, 20, 20, 25), activation='relu') (input1)
model1 = MaxPooling3D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (1, 4, 4, 60))
model2 = Convolution3D(10, 2, 2, 6, input_shape = (1, 4, 4, 60), activation='relu')(input2)
model2 = MaxPooling3D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(1000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

oc = Dense(1,activation='sigmoid', name='label')(bmodel)
oe = Dense(1,activation='sigmoid', name='energy')(bmodel)

model = Model(input=[input1,input2], output=[oc,oe])
model.compile(loss=['binary_crossentropy','mse'], optimizer='sgd')
model.summary()

In [ ]:
class MixedGen:
    #Data generator for regression over energy 
    def __init__( self, batch_size, filepattern='/data/shared/LCD/EnergyScan_Gamma_Shuffled/GammaEscan_*GeV_fulldataset.h5'):
        self.batch_size = batch_size
        self.filelist=[]
        for i in xrange(1,6):
            for j in xrange(1,11):
                self.filelist.append('/data/shared/LCD/New_Data/GammaEscan_%d_%d.h5'%(i,j)) 
        self.train_split = 0.6 
        self.test_split = 0.2 
        self.validation_split = 0.2
        self.fileindex = 0
        self.filesize = 0
        self.position = 0
    #function to call when generating data for training  
    def train(self,modeltype=3):
        length = len(self.filelist)
        #deleting the validation and test set filenames from the filelist
        del self.filelist[np.floor(((self.train_split))*length).astype(int):]
        return self.batches(modeltype)
    #function to call when generating data for testing
    
    #function to call when generating data for validating
    def validation(self,modeltype=3):
        length = len(self.filelist)
        #modifying the filename list to only include files for validation set
        self.filelist = self.filelist[np.floor(self.train_split*length+1).astype(int):np.floor((self.train_split+self.validation_split)*length+1).astype(int)]
        return self.batches(modeltype)
        
    #The function which reads files to gather data until batch size is satisfied
    def batch_helper(self, fileindex, position, batch_size):
        '''
        Yields batches of data of size N
        '''
        f = h5py.File(self.filelist[fileindex],'r')
        self.filesize = np.array(f['ECAL']).shape[0]
        #print(self.filelist[fileindex],'first')
        if (position + batch_size < self.filesize):
            data_ECAL = np.array(f['ECAL'][position : position + batch_size])
            data_HCAL = np.array(f['HCAL'][position : position + batch_size])
            target = np.array(f['target'][position : position + batch_size][:,:,0:2])
            #target = np.delete(target,0,1)

            position += batch_size
            f.close()
            #print('first position',position)
            return data_ECAL,data_HCAL, target, fileindex, position
        
        else:
            data_ECAL = np.array(f['ECAL'][position : ])
            data_HCAL = np.array(f['HCAL'][position : ])
            target = np.array(f['target'][position:][:,:,0:2])
            #target = np.delete(target,0,1)
            f.close()
            
            if (fileindex+1 < len(self.filelist)):
                if(self.batch_size-data_ECAL.shape[0]>0):
                    while(self.batch_size-data_ECAL.shape[0]>0):
                        if(int(np.floor((self.batch_size-data_ECAL.shape[0])/self.filesize))==0):
                            number_of_files=1
                        else:
                            number_of_files=int(np.ceil((self.batch_size-data_ECAL.shape[0])/self.filesize))
                        for i in xrange(0,number_of_files):
                            if(fileindex+i+1>len(self.filelist)):
                                fileindex=0
                                number_of_files=number_of_files-i
                                i=0
                            f = h5py.File(self.filelist[fileindex+i+1],'r')
                            #print(self.filelist[fileindex+i+1],'second')
                            if (self.batch_size-data_ECAL.shape[0]<self.filesize):
                                position = self.batch_size-data_ECAL.shape[0]
                                data_temp_ECAL = np.array(f['ECAL'][ : position])
                                data_temp_HCAL = np.array(f['HCAL'][ : position])
                                target_temp = np.array(f['target'][:position][:,:,0:2])
                            else:
                                data_temp_ECAL = np.array(f['ECAL'])
                                data_temp_HCAL = np.array(f['HCAL'])
                                target_temp = np.array(f['target'][:,:,0:2])
                            f.close()
                    #data_, target_, fileindex, position = self.batch_helper(fileindex + 1, 0, batch_size - self.filesize+position)
                            #print( data.shape,data_.shape)
                            #print( target.shape,target_.shape)
                            data_ECAL = np.concatenate((data_ECAL, data_temp_ECAL), axis=0)
                            data_HCAL = np.concatenate((data_HCAL, data_temp_HCAL), axis=0)
                            target = np.concatenate((target, target_temp), axis=0)
                    if (fileindex +i+1<len(self.filelist)):
                        fileindex = fileindex +i+1
                    else:
                        fileindex = 0
                else:
                    position = 0
                    fileindex=fileindex+1
            else:
                fileindex = 0
                position = 0
            
            return data_ECAL,data_HCAL, target, fileindex, position
    #The function which loops indefinitely and continues to return data of the specified batch size
    def batches(self, modeltype):
        while (self.fileindex < len(self.filelist)):
            data_ECAL,data_HCAL, target, self.fileindex, self.position = self.batch_helper(self.fileindex, self.position, self.batch_size)
            if data_ECAL.shape[0]!=self.batch_size:
                continue
            if modeltype==3:
                data_ECAL = data_ECAL.reshape((data_ECAL.shape[0],)+(1, 24, 24, 25))
                data_HCAL = data_HCAL.reshape((data_HCAL.shape[0],)+(1, 4, 4, 60))
                #data = np.swapaxes(data, 1, 3)
                #data = np.swapaxes(data, 1, 2)
                #data = np.swapaxes(data, 0, 1)
                #data=data.reshape((data.shape[0],1,20,20,25))
            elif modeltype==2:
                data_ECAL = data_ECAL.reshape((data_ECAL.shape[0],)+(24, 24, 25))
                data_ECAL = np.swapaxes(data_ECAL, 1, 3)
                data_HCAL = data_HCAL.reshape((data_HCAL.shape[0],)+(4, 4, 60))
                data_HCAL = np.swapaxes(data_HCAL, 1, 3)
            elif modeltype==1:
                data_ECAL= np.reshape(data_ECAL,(self.batch_size,-1))
                data_HCAL= np.reshape(data_HCAL,(self.batch_size,-1))
            yield ([data_ECAL,data_HCAL],[target[:,:,0],target[:,:,1]/500.])
        self.fileindex = 0
            

In [8]:
class RegGen:
    '''
    Data generator class for directory of h5 files
    '''

    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.filelist=[]
        for i in xrange(1,6):
            for j in xrange(1,11):
                self.filelist.append('/data/shared/LCD/New_Data/GammaEscan_%d_%d.h5'%(i,j)) 
        self.train_split = 0.6
        self.test_split = 0.2
        self.validation_split = 0.2
        self.fileindex = 0
        self.filesize = 0
        self.position = 0

    def train(self,modeltype=3):
        '''
        Generate data for training only
        '''
        length = len(self.filelist)
        #deleting the validation and test set filenames from the filelist
        del self.filelist[int(np.floor((self.train_split) * length)):]
        return self.batches(modeltype)
    
    def validation(self, modeltype=3):
        '''
        Generate data for validation only
        '''
        length = len(self.filelist)
        #modifying the filename list to only include files for validation set
        self.filelist = self.filelist[int(np.floor(self.train_split*length+1)):\
                                      int(np.floor((self.train_split + self.validation_split) * length+1))]
        return self.batches(modeltype)
        
    #The function which reads files to gather data until batch size is satisfied
    def batch_helper(self, fileindex, position, batch_size):
        '''
        Reads files to gather data until batch size is satisfied, then yeilds
        '''
        f = h5py.File(self.filelist[fileindex], 'r')
        self.filesize = np.array(f['ECAL']).shape[0]
        #print(self.filelist[fileindex],'first')
        if (position + batch_size < self.filesize):
            data_ECAL = np.array(f['ECAL'][position : position + batch_size])
            data_HCAL = np.array(f['HCAL'][position : position + batch_size])
            target  = np.array(f['target'][position : position + batch_size][:,:,1])
            #target = np.delete(target,0,1)

            position += batch_size
            f.close()
            #print('first position',position)
            return data_ECAL, data_HCAL, target, fileindex, position
        
        else:
            data_ECAL = np.array(f['ECAL'][position:])
            data_HCAL = np.array(f['HCAL'][position:])
            target =  np.array(f['target'][position:][:,:,1])
            #target = np.delete(target,0,1)
            f.close()
            
            if fileindex+1 < len(self.filelist):
                if self.batch_size - data_ECAL.shape[0] > 0:
                    while self.batch_size - data_ECAL.shape[0] > 0:

                        if int(np.floor((self.batch_size - data_ECAL.shape[0]) / self.filesize)) == 0:
                            number_of_files = 1
                        else:
                            number_of_files = int(np.ceil((self.batch_size-data_ECAL.shape[0]) / self.filesize))

                        for i in xrange(0, number_of_files):
                            # restart in file list in case we run out of files
                            if fileindex + i + 1 > len(self.filelist):
                                fileindex = -1 - i

                            f = h5py.File(self.filelist[fileindex+i+1],'r')
                            #print(self.filelist[fileindex+i+1],'second')
                            if (self.batch_size - data_ECAL.shape[0] < self.filesize):
                                position = self.batch_size - data_ECAL.shape[0]
                                data_temp_ECAL = np.array(f['ECAL'][:position])
                                data_temp_HCAL = np.array(f['HCAL'][:position])
                                target_temp = np.array(f['target'][:position][:,:,1])

                            else:
                                data_temp_ECAL = np.array(f['ECAL'])
                                data_temp_HCAL = np.array(f['HCAL'])
                                target_temp = np.array(f['target'][:,:,1])

                            f.close()
                            #print( data.shape,data_.shape)
                            #print( target.shape,target_.shape)
                            data_ECAL = np.concatenate((data_ECAL, data_temp_ECAL), axis=0)
                            data_HCAL = np.concatenate((data_HCAL, data_temp_HCAL), axis=0)
                            target = np.concatenate((target, target_temp), axis=0)
                    
                    if (fileindex + i + 1 < len(self.filelist)):
                        fileindex += i + 1
                    else:
                        fileindex = 0
                else:
                    position = 0
                    fileindex += 1
            else:
                fileindex = 0
                position = 0
            
            return data_ECAL, data_HCAL, target, fileindex, position
    
    def batches(self, modeltype):
        '''
        Loops indefinitely and continues to return data of specified batch size
        '''
        while (self.fileindex < len(self.filelist)):
            data_ECAL,data_HCAL, target, self.fileindex, self.position = self.batch_helper(self.fileindex, self.position, self.batch_size)
            if data_ECAL.shape[0]!=self.batch_size:
                continue
            if modeltype==3:
                data_ECAL = data_ECAL.reshape((data_ECAL.shape[0],)+(1, 24, 24, 25))
                data_HCAL = data_HCAL.reshape((data_HCAL.shape[0],)+(1, 4, 4, 60))
                #data = np.swapaxes(data, 1, 3)
                #data = np.swapaxes(data, 1, 2)
                #data = np.swapaxes(data, 0, 1)
                #data=data.reshape((data.shape[0],1,20,20,25))
            elif modeltype==2:
                data_ECAL = data_ECAL.reshape((data_ECAL.shape[0],)+(24, 24, 25))
                data_ECAL = np.swapaxes(data_ECAL, 1, 3)
                data_HCAL = data_HCAL.reshape((data_HCAL.shape[0],)+(4, 4, 60))
                data_HCAL = np.swapaxes(data_HCAL, 1, 3)
            elif modeltype==1:
                data_ECAL= np.reshape(data_ECAL,(self.batch_size,-1))
                data_HCAL= np.reshape(data_HCAL,(self.batch_size,-1))
            yield ([data_ECAL,data_HCAL],target/500.)
        self.fileindex = 0


In [ ]:
class ClsGen:
    #Data generator for regression over energy 
    def __init__( self, batch_size, filepattern='/data/shared/LCD/EnergyScan_Gamma_Shuffled/GammaEscan_*GeV_fulldataset.h5'):
        self.batch_size = batch_size
        self.filelist=[]
        for i in xrange(1,6):
            for j in xrange(1,11):
                self.filelist.append('/data/shared/LCD/New_Data/GammaEscan_%d_%d.h5'%(i,j)) 
        self.train_split = 0.6 
        self.test_split = 0.2 
        self.validation_split = 0.2
        self.fileindex = 0
        self.filesize = 0
        self.position = 0
    #function to call when generating data for training  
    def train(self,modeltype=3):
        length = len(self.filelist)
        #deleting the validation and test set filenames from the filelist
        del self.filelist[np.floor(((self.train_split))*length).astype(int):]
        return self.batches(modeltype)
    #function to call when generating data for testing
   
    #function to call when generating data for validating
    def validation(self,modeltype=3):
        length = len(self.filelist)
        #modifying the filename list to only include files for validation set
        self.filelist = self.filelist[np.floor(self.train_split*length+1).astype(int):np.floor((self.train_split+self.validation_split)*length+1).astype(int)]
        return self.batches(modeltype)
        
    #The function which reads files to gather data until batch size is satisfied
    def batch_helper(self, fileindex, position, batch_size):
        '''
        Yields batches of data of size N
        '''
        f = h5py.File(self.filelist[fileindex],'r')
        self.filesize = np.array(f['ECAL']).shape[0]
        #print(self.filelist[fileindex],'first')
        if (position + batch_size < self.filesize):
            data_ECAL = np.array(f['ECAL'][position : position + batch_size])
            data_HCAL = np.array(f['HCAL'][position : position + batch_size])
            target = np.array(f['target'][position : position + batch_size][:,:,0])
            #target = np.delete(target,0,1)

            position += batch_size
            f.close()
            #print('first position',position)
            return data_ECAL,data_HCAL, target, fileindex, position
        
        else:
            data_ECAL = np.array(f['ECAL'][position : ])
            data_HCAL = np.array(f['HCAL'][position : ])
            target = np.array(f['target'][position:][:,:,0])
            #target = np.delete(target,0,1)
            f.close()
            
            if (fileindex+1 < len(self.filelist)):
                if(self.batch_size-data_ECAL.shape[0]>0):
                    while(self.batch_size-data_ECAL.shape[0]>0):
                        if(int(np.floor((self.batch_size-data_ECAL.shape[0])/self.filesize))==0):
                            number_of_files=1
                        else:
                            number_of_files=int(np.ceil((self.batch_size-data_ECAL.shape[0])/self.filesize))
                        for i in xrange(0,number_of_files):
                            if(fileindex+i+1>len(self.filelist)):
                                fileindex=0
                                number_of_files=number_of_files-i
                                i=0
                            f = h5py.File(self.filelist[fileindex+i+1],'r')
                            #print(self.filelist[fileindex+i+1],'second')
                            if (self.batch_size-data_ECAL.shape[0]<self.filesize):
                                position = self.batch_size-data_ECAL.shape[0]
                                data_temp_ECAL = np.array(f['ECAL'][ : position])
                                data_temp_HCAL = np.array(f['HCAL'][ : position])
                                target_temp = np.array(f['target'][:position][:,:,0])
                            else:
                                data_temp_ECAL = np.array(f['ECAL'])
                                data_temp_HCAL = np.array(f['HCAL'])
                                target_temp = np.array(f['target'][:,:,0])
                            f.close()
                    #data_, target_, fileindex, position = self.batch_helper(fileindex + 1, 0, batch_size - self.filesize+position)
                            #print( data.shape,data_.shape)
                            #print( target.shape,target_.shape)
                            data_ECAL = np.concatenate((data_ECAL, data_temp_ECAL), axis=0)
                            data_HCAL = np.concatenate((data_HCAL, data_temp_HCAL), axis=0)
                            target = np.concatenate((target, target_temp), axis=0)
                    if (fileindex +i+1<len(self.filelist)):
                        fileindex = fileindex +i+1
                    else:
                        fileindex = 0
                else:
                    position = 0
                    fileindex=fileindex+1
            else:
                fileindex = 0
                position = 0
            
            return data_ECAL,data_HCAL, target, fileindex, position
    #The function which loops indefinitely and continues to return data of the specified batch size
    def batches(self, modeltype):
        while (self.fileindex < len(self.filelist)):
            data_ECAL,data_HCAL, target, self.fileindex, self.position = self.batch_helper(self.fileindex, self.position, self.batch_size)
            if data_ECAL.shape[0]!=self.batch_size:
                continue
            if modeltype==3:
                data_ECAL = data_ECAL.reshape((data_ECAL.shape[0],)+(1, 24, 24, 25))
                data_HCAL = data_HCAL.reshape((data_HCAL.shape[0],)+(1, 4, 4, 60))
                #data = np.swapaxes(data, 1, 3)
                #data = np.swapaxes(data, 1, 2)
                #data = np.swapaxes(data, 0, 1)
                #data=data.reshape((data.shape[0],1,20,20,25))
            elif modeltype==2:
                data_ECAL = data_ECAL.reshape((data_ECAL.shape[0],)+(24, 24, 25))
                data_ECAL = np.swapaxes(data_ECAL, 1, 3)
                data_HCAL = data_HCAL.reshape((data_HCAL.shape[0],)+(4, 4, 60))
                data_HCAL = np.swapaxes(data_HCAL, 1, 3)
            elif modeltype==1:
                data_ECAL= np.reshape(data_ECAL,(self.batch_size,-1))
                data_HCAL= np.reshape(data_HCAL,(self.batch_size,-1))
            yield ([data_ECAL,data_HCAL],target)
        self.fileindex = 0
            

In [9]:
model = loadmodel('Models_CSCS/bcnn1_reg')
model.compile(loss=['mse'],optimizer='sgd')
model.summary()

Loaded model from disk
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1, 24, 24, 25) 0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 1, 4, 4, 60)   0                                            
____________________________________________________________________________________________________
convolution3d_1 (Convolution3D)  (None, 10, 21, 21, 21)810         input_1[0][0]                    
____________________________________________________________________________________________________
convolution3d_2 (Convolution3D)  (None, 10, 3, 3, 55)  250         input_2[0][0]                    
____________________________________________________________________

In [10]:
ds1 = RegGen(500)
vs1 = RegGen(500)
#check = ModelCheckpoint(filepath="temp.hdf5", verbose=1)
#early = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
hist = model.fit_generator(ds1.train(modeltype=3), samples_per_epoch=1000, nb_epoch=50, validation_data= vs1.validation(modeltype=3), nb_val_samples=1000, verbose=1)

Epoch 1/50
1000/1000 [==============================] - 67s - loss: 0.1073 - val_loss: 0.0917
Epoch 2/50
1000/1000 [==============================] - 59s - loss: 0.1063 - val_loss: 0.0960
Epoch 3/50
1000/1000 [==============================] - 52s - loss: 0.1003 - val_loss: 0.0879
Epoch 4/50
1000/1000 [==============================] - 30s - loss: 0.1055 - val_loss: 0.0863
Epoch 5/50
1000/1000 [==============================] - 30s - loss: 0.1001 - val_loss: 0.0844
Epoch 6/50
1000/1000 [==============================] - 30s - loss: 0.1023 - val_loss: 0.0895
Epoch 7/50
1000/1000 [==============================] - 31s - loss: 0.1029 - val_loss: 0.0875
Epoch 8/50
1000/1000 [==============================] - 30s - loss: 0.1041 - val_loss: 0.0838
Epoch 9/50
1000/1000 [==============================] - 31s - loss: 0.0930 - val_loss: 0.0877
Epoch 10/50
1000/1000 [==============================] - 31s - loss: 0.0972 - val_loss: 0.0876
Epoch 11/50
1000/1000 [==============================] - 34

In [ ]:
#Dense Model
input1 = Input(shape = (14400,))
model1 = Dense(10000, activation='relu') (input1)
model1 = Dense(10000, activation='relu') (model1)

input2 = Input(shape = (960,))
model2 = Dense(10000,activation='relu')(input2)
model2 = Dense(10000, activation='relu')(model2)


## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='sigmoid')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='sigmoid')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='sigmoid')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
oe = Dense(1,activation='sigmoid', name='energy')(bmodel)

model = Model(input=[input1,input2], output=[oc,oe])
model.compile(loss=['binary_crossentropy','mse'], optimizer='sgd')
model.summary()

In [ ]:
#Regression Model
input1 = Input(shape = (25,24,24))
model1 = Convolution2D(10,4, 4,input_shape = (25,24,24), activation='relu') (input1)
model1 = Convolution2D(10,3, 3, activation='relu') (model1)
model1 = MaxPooling2D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (60,4,4))
model2 = Convolution2D(10, 2, 2,input_shape = (60,4,4), activation='relu')(input2)
model2 = Convolution2D(3, 2, 2, activation='relu')(model2)
model2 = MaxPooling2D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='sigmoid')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='sigmoid')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

#bmodel = (Dense(10, activation='sigmoid')) (bmodel)
#bmodel = (Dropout(0.5)) (bmodel)

#oc = Dense(1,activation='sigmoid', name='particleID')(bmodel)
oe = Dense(1,activation='sigmoid', name='energy')(bmodel)

model = Model(input=[input1,input2], output=[oe])
model.compile(loss=['mse'], optimizer='sgd')
model.summary()

In [ ]:
savemodel(model,'dense1_regcl')

In [ ]:
ds1 = RegGenFull(5000, 10000)
vs1 = RegGenFull(5000, 10000)
early = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
hist = model.fit_generator(ds1.train(cnn=True), samples_per_epoch=50000, nb_epoch=30, validation_data= vs1.validation(cnn=True), nb_val_samples=50000, verbose=1, callbacks=[early])

In [ ]:
savemodel(model,'bcnn_1')

In [ ]:
savelosses(hist,'bcnn_1')

In [ ]:
#savemodel(cnn2d_1,"cnn2d_1")
%matplotlib inline
show_losses([("mse",hist)],"bcnn_1")

In [ ]:
def convert(filename):
    f = h5py.File(filename,'r')
    HCAL = np.array(f['HCAL'])
    ECAL = np.array(f['ECAL'])
    target=np.array(f['target'])
    f.close()
    np.savez_compressed(filename.replace('.h5','.npz'), HCAL=HCAL,ECAL=ECAL,target=target)

In [ ]:
for i in xrange(1,101):
    print(i)
    convert('/data/shared/LCD/New_Data_Shuffled/GammaPi0_shuffled_%d.h5'%i)

In [ ]:
json_file = open('bcnn_reg_4.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.summary()

In [ ]:
json_file = open('models/bcnn_cl_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.summary()

In [ ]:
model = loadmodel('models/bcnn_cl_1',weights=False)

In [ ]:
model.summary()

In [ ]:
import numpy as np
import os
import sys
import re
import glob
import h5py
import numpy as np
#import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Activation,Input, Dense, Dropout, merge
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the models' predictions and histories
#predictions = {}
#histories = {}
model_names = []
mc = 0
for l in filter(None, os.popen('ls models/*.json').read().split('\n')):
    #m_name = l.split('/')[-1].split('.')[0]
    l = l.split('.')[0]
    print (mc+1, l)
    mc = mc+1 
    model_names.append(l)   
    # Load model information
    model = loadmodel(l, weights=False)
    print (m_name)
    model.summary()

In [ ]:
filename = 'bcnn_1_reg.json'
print(filename.replace('.json','')+'.h5')

In [ ]:
filename = 'Models_CSCS/dense1_cl'
ds = ClsGen(500)
vs = ClsGen(500)
model = loadmodel(filename)
model.compile(loss=['binary_crossentropy'],optimizer='sgd')
check = ModelCheckpoint(filepath="temp.h5", verbose=0)
early = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
print('%s has started training'%filename)
hist = model.fit_generator(ds.train(modeltype=1), samples_per_epoch=500, nb_epoch=1, validation_data= vs.validation(modeltype=1), nb_val_samples=500, verbose=0, callbacks=[check,early])
savelosses(hist,name=filename.replace('.json',''))
savemodel(model,name=filename.replace('.json',''))
print('%s has finished training'%filename)

In [ ]:
def modelsave(model,name="neural network"):

    model_name = name
    print name
    model.summary()
    #model.save_weights('%s.h5'%model_name, overwrite=True)
    model_json = model.to_json()
    with open('models/'+"%s.json"%model_name, "w") as json_file:
        json_file.write(model_json)

In [ ]:
#All same topologies as above, but with relu instead of sigmoid

#Regression Models 

##Model 1
input1 = Input(shape = (1, 24, 24, 25))
model1 = Convolution3D(10, 4, 4, 5, input_shape = (1, 24, 24, 25), activation='relu') (input1)
model1 = MaxPooling3D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (1, 4, 4, 60))
model2 = Convolution3D(10, 2, 2, 6, input_shape = (1, 4, 4, 60), activation='relu')(input2)
model2 = MaxPooling3D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(10, activation='relu')) (bmodel)

#oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
oe = Dense(1,activation='linear', name='energy')(bmodel)

bcnnr1 = Model(input=[input1,input2], output=[oe])
bcnnr1.compile(loss=['mse'], optimizer='sgd')
bcnnr1.summary()



#Regression Models 

##Model 2
input1 = Input(shape = (1, 24, 24, 25))
model1 = Convolution3D(10, 4, 4, 5, input_shape = (1, 24, 24, 25), activation='relu') (input1)
model1 = MaxPooling3D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (1, 4, 4, 60))
model2 = Convolution3D(10, 2, 2, 6, input_shape = (1, 4, 4, 60), activation='relu')(input2)
model2 = MaxPooling3D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

#oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
oe = Dense(1,activation='linear', name='energy')(bmodel)

bcnnr2 = Model(input=[input1,input2], output=[oe])
bcnnr2.compile(loss=['mse'], optimizer='sgd')
bcnnr2.summary()


#Regression Models 

##Model 3
input1 = Input(shape = (1, 24, 24, 25))
model1 = Convolution3D(10, 4, 4, 5, input_shape = (1, 24, 24, 25), activation='relu') (input1)
model1 = MaxPooling3D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (1, 4, 4, 60))
model2 = Convolution3D(10, 2, 2, 6, input_shape = (1, 4, 4, 60), activation='relu')(input2)
model2 = MaxPooling3D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

#oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
oe = Dense(1,activation='linear', name='energy')(bmodel)

bcnnr3 = Model(input=[input1,input2], output=[oe])
bcnnr3.compile(loss=['mse'], optimizer='sgd')
bcnnr3.summary()

#Regression Models 

##Model 4
input1 = Input(shape = (1, 24, 24, 25))
model1 = Convolution3D(10, 4, 4, 5, input_shape = (1, 24, 24, 25), activation='relu') (input1)
model1 = Convolution3D(3, 2, 2, 5, input_shape = (1, 24, 24, 25), activation='relu') (model1)
model1 = MaxPooling3D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (1, 4, 4, 60))
model2 = Convolution3D(10, 2, 2, 6, input_shape = (1, 4, 4, 60), activation='relu')(input2)
model2 = Convolution3D(3, 2, 2, 5, input_shape = (1, 4, 4, 60), activation='relu')(model2)
model2 = MaxPooling3D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(10, activation='relu')) (bmodel)

#oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
oe = Dense(1,activation='linear', name='energy')(bmodel)

bscnnr1 = Model(input=[input1,input2], output=[oe])
bscnnr1.compile(loss=['mse'], optimizer='sgd')
bscnnr1.summary()

#Regression Models 

##Model 5
input1 = Input(shape = (25,24,24))
model1 = Convolution2D(10, 4, 4, input_shape = (25,24,24), activation='relu') (input1)
model1 = Convolution2D(3, 2, 2,activation='relu') (model1)
model1 = MaxPooling2D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (60,4,4))
model2 = Convolution2D(10, 2, 2,input_shape = (60,4,4), activation='relu')(input2)
model2 = Convolution2D(3, 2, 2,activation='relu')(model2)
model2 = MaxPooling2D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(10, activation='relu')) (bmodel)

#oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
oe = Dense(1,activation='linear', name='energy')(bmodel)

bscnnr2 = Model(input=[input1,input2], output=[oe])
bscnnr2.compile(loss=['mse'], optimizer='sgd')
bscnnr2.summary()


#Dense Model
input1 = Input(shape = (14400,))
model1 = Dense(10000, activation='relu') (input1)
model1 = Dense(10000, activation='relu') (model1)

input2 = Input(shape = (960,))
model2 = Dense(10000,activation='relu')(input2)
model2 = Dense(10000, activation='relu')(model2)


## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

#oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
oe = Dense(1,activation='linear', name='energy')(bmodel)

denser = Model(input=[input1,input2], output=[oe])
denser.compile(loss=['mse'], optimizer='sgd')
denser.summary()


modelsave(bscnnr1,"bcnnr4_reg")
modelsave(bscnnr2,"bcnnr5_reg")
modelsave(bcnnr1,"bcnnr1_regc")
modelsave(bcnnr2,"bcnnr2_reg")
modelsave(bcnnr3,"bcnnr3_reg")
#draw_model(bcnnr3, "bcnnr5_reg")
modelsave(denser,'denser1_reg')

In [ ]:
#All same topologies as above, but with relu instead of sigmoid

#Classification Models 

##Model 1
input1 = Input(shape = (1, 24, 24, 25))
model1 = Convolution3D(10, 4, 4, 5, input_shape = (1, 24, 24, 25), activation='relu') (input1)
model1 = MaxPooling3D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (1, 4, 4, 60))
model2 = Convolution3D(10, 2, 2, 6, input_shape = (1, 4, 4, 60), activation='relu')(input2)
model2 = MaxPooling3D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(10, activation='relu')) (bmodel)

oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
#oe = Dense(1,activation='linear', name='energy')(bmodel)

bcnnr1 = Model(input=[input1,input2], output=[oc])
bcnnr1.compile(loss=['binary_crossentropy'], optimizer='sgd')
bcnnr1.summary()



#Classification Models 

##Model 2
input1 = Input(shape = (1, 24, 24, 25))
model1 = Convolution3D(10, 4, 4, 5, input_shape = (1, 24, 24, 25), activation='relu') (input1)
model1 = MaxPooling3D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (1, 4, 4, 60))
model2 = Convolution3D(10, 2, 2, 6, input_shape = (1, 4, 4, 60), activation='relu')(input2)
model2 = MaxPooling3D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
#oe = Dense(1,activation='linear', name='energy')(bmodel)

bcnnr2 = Model(input=[input1,input2], output=[oc])
bcnnr2.compile(loss=['binary_crossentropy'], optimizer='sgd')
bcnnr2.summary()


#Classification Models 

##Model 3
input1 = Input(shape = (1, 24, 24, 25))
model1 = Convolution3D(10, 4, 4, 5, input_shape = (1, 24, 24, 25), activation='relu') (input1)
model1 = MaxPooling3D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (1, 4, 4, 60))
model2 = Convolution3D(10, 2, 2, 6, input_shape = (1, 4, 4, 60), activation='relu')(input2)
model2 = MaxPooling3D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
#oe = Dense(1,activation='linear', name='energy')(bmodel)

bcnnr3 = Model(input=[input1,input2], output=[oc])
bcnnr3.compile(loss=['binary_crossentropy'], optimizer='sgd')
bcnnr3.summary()

#Classification Models 

##Model 4
input1 = Input(shape = (1, 24, 24, 25))
model1 = Convolution3D(10, 4, 4, 5, input_shape = (1, 24, 24, 25), activation='relu') (input1)
model1 = Convolution3D(3, 2, 2, 5, input_shape = (1, 24, 24, 25), activation='relu') (model1)
model1 = MaxPooling3D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (1, 4, 4, 60))
model2 = Convolution3D(10, 2, 2, 6, input_shape = (1, 4, 4, 60), activation='relu')(input2)
model2 = Convolution3D(3, 2, 2, 5, input_shape = (1, 4, 4, 60), activation='relu')(model2)
model2 = MaxPooling3D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(10, activation='relu')) (bmodel)

oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
#oe = Dense(1,activation='linear', name='energy')(bmodel)

bscnnr1 = Model(input=[input1,input2], output=[oc])
bscnnr1.compile(loss=['binary_crossentropy'], optimizer='sgd')
bscnnr1.summary()

#Classification Models 

##Model 5
input1 = Input(shape = (25,24,24))
model1 = Convolution2D(10, 4, 4, input_shape = (25,24,24), activation='relu') (input1)
model1 = Convolution2D(3, 2, 2,activation='relu') (model1)
model1 = MaxPooling2D()(model1)
model1 = Flatten()(model1)

input2 = Input(shape = (60,4,4))
model2 = Convolution2D(10, 2, 2,input_shape = (60,4,4), activation='relu')(input2)
model2 = Convolution2D(3, 2, 2,activation='relu')(model2)
model2 = MaxPooling2D()(model2)
model2 = Flatten()(model2)

## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(10, activation='relu')) (bmodel)

oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
#oe = Dense(1,activation='linear', name='energy')(bmodel)

bscnnr2 = Model(input=[input1,input2], output=[oc])
bscnnr2.compile(loss=['binary_crossentropy'], optimizer='sgd')
bscnnr2.summary()


#Dense Model
input1 = Input(shape = (14400,))
model1 = Dense(10000, activation='relu') (input1)
model1 = Dense(10000, activation='relu') (model1)

input2 = Input(shape = (960,))
model2 = Dense(10000,activation='relu')(input2)
model2 = Dense(10000, activation='relu')(model2)


## join the two
bmodel = merge([model1,model2], mode='concat')

## fully connected ending
bmodel = (Dense(10000, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

bmodel = (Dense(100, activation='relu')) (bmodel)
bmodel = (Dropout(0.5)) (bmodel)

oc = Dense(1,activation='sigmoid', name='particle label')(bmodel)
#oe = Dense(1,activation='linear', name='energy')(bmodel)

denser = Model(input=[input1,input2], output=[oc])
denser.compile(loss=['binary_crossentropy'], optimizer='sgd')
denser.summary()


modelsave(bscnnr1,"bcnnr4_cl")
modelsave(bscnnr2,"bcnnr5_cl")
modelsave(bcnnr1,"bcnnr1_cl")
modelsave(bcnnr2,"bcnnr2_cl")
modelsave(bcnnr3,"bcnnr3_cl")
#draw_model(bcnnr3, "bcnnr5_regcl")
modelsave(denser,'denser1_cl')